In [1]:
import numpy as np
import pandas as pd

In [5]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [7]:
df = pd.read_csv("train.csv", usecols=['Age','Fare','Survived'])

In [16]:
df.dropna(inplace = True)

In [18]:
df.shape

(714, 3)

In [20]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [28]:
x = df.iloc[:,1:]

In [32]:
y = df.iloc[:,0]

In [34]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2,random_state = 42)

In [44]:
x_test.shape

(143, 2)

In [46]:
clf = DecisionTreeClassifier()

In [48]:
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [50]:
accuracy_score(y_test,y_pred)

0.6363636363636364

In [58]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv = 10, scoring = 'accuracy'))

0.6330985915492958

In [82]:
kbin_age = KBinsDiscretizer(n_bins = 5, encode = 'ordinal', strategy = 'quantile')
kbin_fare = KBinsDiscretizer(n_bins = 5, encode = 'ordinal', strategy = 'quantile')

In [84]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [86]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [94]:
trf.named_transformers_['first'].n_bins_

array([5])

In [98]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train_trf[:,0],
    'fare': x_train['Fare'],
    'Fare_trf':x_train_trf[:,1]
})

In [108]:
output['age_label'] = pd.cut(x = x_train['Age'],bins = trf.named_transformers_['first'].bin_edges_[0].tolist())

In [116]:
output['fare_labels'] = pd.cut(x = x_train['Fare'], bins = trf.named_transformers_['second'].bin_edges_[0].tolist())

In [118]:
output.sample(5)

,age,age_trf,fare,Fare_trf,age_label,fare_labels
292,36.0,3.0,12.8750,1.0,"(32.0, 42.0]","(7.896, 13.0]"
156,16.0,0.0,7.7333,0.0,"(0.42, 19.0]","(0.0, 7.896]"
453,49.0,4.0,89.1042,4.0,"(42.0, 80.0]","(51.479, 512.329]"
814,30.5,2.0,8.0500,1.0,"(25.0, 32.0]","(7.896, 13.0]"
344,36.0,3.0,13.0000,2.0,"(32.0, 42.0]","(7.896, 13.0]"


In [120]:
clf= DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf)

In [122]:
accuracy_score(y_test,y_pred2)

0.6433566433566433

In [144]:
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),x_trf,y,cv = 10,scoring = 'accuracy'))

0.673943661971831

In [146]:
def discretize(bins,strategy):
    kbin_age = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    kbin_fare = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    
    trf = ColumnTransformer([
        ('first',kbin_age,[0]),
        ('second',kbin_fare,[1])
    ])
    
    x_trf = trf.fit_transform(x)
    print(np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy')))
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(x['Age'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(x_trf[:,0],color='red')
    plt.title("After")

    plt.show()
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(x['Fare'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(x_trf[:,1],color='red')
    plt.title("Fare")

    plt.show()
    
